# Host-guest usage with stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

#### Using latest version of `SpinDry` and `stk`.

- Changes in `SpinDry.Molecule` means we now use `spd.Molecule()` not `spd.Molecule.init()`
- `spd.Spinner.get_conformers` should now take `spd.Supramolecule`, which we initialise from components.
- To then access those components, you must do that through the `Supramolecule` interface.
- Finally, we had to change to using the `stk.complex.Guest` interface.
- See https://stk.readthedocs.io/en/stable/_autosummary/stk.host_guest.Complex.html
- See https://github.com/andrewtarzia/SpinDry/blob/main/examples/

# Imports 

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import os
import time
import stk
import stko
import spindry as spd
%matplotlib inline

# Some useful functions 

In [ ]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

Working:

In [ ]:
bb1 = stk.BuildingBlock(
    smiles='O=CC(C=O)C=O',
    functional_groups=[stk.AldehydeFactory()],
)
bb2 = stk.BuildingBlock(
    smiles='O=CC(Cl)(C=O)C=O',
    functional_groups=[stk.AldehydeFactory()],
)
bb3 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb4 = stk.BuildingBlock(
    smiles='NCC(Cl)N',
    functional_groups=[stk.PrimaryAminoFactory()],
)
bb5 = stk.BuildingBlock('NCCCCN', [stk.PrimaryAminoFactory()])

cage = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix(
        # building_blocks is now a dict, which maps building
        # blocks to the id of the vertices it should be placed
        # on. You can use ranges to specify the ids.
        building_blocks={
            bb1: range(2),
            bb2: (2, 3),
            bb3: 4,
            bb4: 5,
            bb5: range(6, 10),
        },
        optimizer=stk.MCHammer(),
    ),
)

In [ ]:
show_stk_mol(cage)

In [ ]:
cage_energy = stko.MMFFEnergy(ignore_inter_interactions=False).get_energy(cage)

In [ ]:
print(cage_energy)

In [ ]:
stk_guest = stk.BuildingBlock('C1CC=CCC(C1)O')

In [ ]:
guest_energy = stko.MMFFEnergy(ignore_inter_interactions=False).get_energy(stk_guest)

In [ ]:
print(guest_energy)

In [ ]:
show_stk_mol(stk_guest)

In [ ]:
cage = stk.BuildingBlock.init_from_molecule(cage)
cage_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in cage.get_atoms()
]

# Build stk guest.
guest_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in stk_guest.get_atoms()
]

host = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in cage_atoms
    ),
    bonds=(),
    position_matrix=cage.get_position_matrix(),
)
guest = spd.Molecule(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in guest_atoms
    ),
    bonds=(),
    position_matrix=stk_guest.get_position_matrix(),
)

In [ ]:
host

In [ ]:
guest

In [ ]:
cg = spd.Spinner(
    step_size=0.5,
    rotation_step_size=5,
    num_conformers=100,
    max_attempts=10000,
)

In [ ]:
cg

In [ ]:
supramolecule = spd.SupraMolecule.init_from_components(components=(host, guest))

In [ ]:
binding_energies = {}

for conformer in cg.get_conformers(supramolecule):
    print(conformer)
    print(conformer.get_cid(), conformer.get_potential())
    for i, comp in enumerate(list(conformer.get_components())):
        if i == 0:
            cage = cage.with_position_matrix(comp.get_position_matrix())
        elif i == 1:
            bound_guest = stk_guest.with_position_matrix(comp.get_position_matrix())
            
    complex_mol = stk.ConstructedMolecule(
        topology_graph=stk.host_guest.Complex(host=cage, guests=stk.host_guest.Guest(bound_guest))
    )
    complex_mol.write(
        f'hg_example_output/conf_{conformer.get_cid()}.mol'
    )
    bound_energy = stko.MMFFEnergy(ignore_inter_interactions=False).get_energy(complex_mol)
    binding_energy = bound_energy - (cage_energy + guest_energy)
    binding_energies[conformer.get_cid()] = binding_energy

In [ ]:
print(binding_energies)

In [ ]:
plt.hist([binding_energies[i] for i in binding_energies])
plt.xlabel('binding energy []')
plt.show()